# Time Series (beta) Pycaret

Imports

In [2]:
import pycaret

In [2]:
import matplotlib.pyplot as plt 
from matplotlib.pylab import rcParams
import pandas as pd
import scipy.stats as sts
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid',font='Arial',font_scale=1.2)
rcParams['figure.figsize'] = 12,8
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

# Dataset from zillow api 

* Dataset was cleaned in [EDA.ipynb]('EDA.ipynb')
* cook county IL filtered
* Values = mean price change over time
* index = neighborhoods (cook county IL only!!)
* columns = time = (YYYY/MM/DD) min=2000-01-31 / max=2022-02-28
* in this test we will test `chicago` individially as a beta test and determine the value of continuing with this method

In [11]:
#import dataset
dataset = pd.read_csv('Resources/cookcounty.csv',index_col='RegionName')

In [12]:
dataset

,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,...,2021-05-31,2021-06-30,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28
RegionName,,,,,,,,,,,,,,,,,,,,,
Chicago,165282.0,165621.0,166381.0,167702.0,169196.0,170566.0,172001.0,173594.0,175129.0,176760.0,...,278732.0,281106.0,283652.0,287049.0,289393.0,291647.0,294440.0,297088.0,299729.0,301283.0
Cicero,128972.0,129220.0,129706.0,130518.0,131517.0,132603.0,133856.0,135013.0,136012.0,137035.0,...,226907.0,229498.0,232578.0,235283.0,237289.0,238760.0,240861.0,243257.0,245957.0,247645.0
Arlington Heights,261429.0,261759.0,262735.0,264406.0,266390.0,267808.0,269428.0,271458.0,273819.0,276055.0,...,390213.0,394921.0,400663.0,405699.0,408596.0,411208.0,414526.0,417568.0,420270.0,421782.0
Schaumburg,206420.0,206899.0,207706.0,208993.0,210318.0,211430.0,213118.0,215316.0,217632.0,219828.0,...,318749.0,322037.0,325884.0,329969.0,332546.0,335233.0,338328.0,341091.0,343829.0,345585.0
Evanston,302592.0,302905.0,303645.0,304993.0,306863.0,308430.0,310211.0,312727.0,316042.0,320664.0,...,502345.0,509733.0,518375.0,526181.0,532433.0,538210.0,544713.0,550440.0,554475.0,556319.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Golf,511254.0,512519.0,515788.0,520828.0,524009.0,527693.0,529683.0,535512.0,537426.0,545143.0,...,812506.0,837445.0,865139.0,879458.0,890257.0,898739.0,909127.0,919617.0,929894.0,935990.0
Hodgkins,146250.0,144877.0,145472.0,146342.0,148481.0,149861.0,150805.0,151201.0,152082.0,153320.0,...,289536.0,292275.0,296267.0,299403.0,302809.0,304813.0,307362.0,308740.0,310763.0,311582.0
McCook,164992.0,162667.0,162618.0,162454.0,162897.0,163778.0,164053.0,165607.0,166947.0,168724.0,...,279015.0,282892.0,288192.0,293391.0,297558.0,301130.0,304551.0,307136.0,309673.0,310853.0


In [13]:
#Transpose data
dataset = dataset.T

In [14]:
dataset

RegionName,Chicago,Cicero,Arlington Heights,Schaumburg,Evanston,Orland Park,Palatine,Des Plaines,Skokie,Oak Lawn,...,Kenilworth,Bedford Park,East Hazel Crest,Phoenix,Merrionette Park,Golf,Hodgkins,McCook,Forestview,Great Lakes
2000-01-31,165282.0,128972.0,261429.0,206420.0,302592.0,241080.0,235324.0,195088.0,239853.0,164582.0,...,916906.0,129734.0,118766.0,71375.0,137084.0,511254.0,146250.0,164992.0,164312.0,NaN
2000-02-29,165621.0,129220.0,261759.0,206899.0,302905.0,241595.0,235879.0,195468.0,240143.0,164751.0,...,920207.0,130722.0,118536.0,71113.0,137635.0,512519.0,144877.0,162667.0,163407.0,NaN
2000-03-31,166381.0,129706.0,262735.0,207706.0,303645.0,242463.0,236957.0,196411.0,240978.0,165133.0,...,923508.0,131705.0,118846.0,71177.0,137944.0,515788.0,145472.0,162618.0,165091.0,NaN
2000-04-30,167702.0,130518.0,264406.0,208993.0,304993.0,243992.0,238737.0,197934.0,242524.0,165835.0,...,933200.0,133287.0,119151.0,71259.0,138793.0,520828.0,146342.0,162454.0,166567.0,NaN
2000-05-31,169196.0,131517.0,266390.0,210318.0,306863.0,245510.0,240503.0,199537.0,244546.0,166470.0,...,943445.0,134495.0,119125.0,70928.0,139376.0,524009.0,148481.0,162897.0,168339.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-31,291647.0,238760.0,411208.0,335233.0,538210.0,364067.0,351596.0,320336.0,380049.0,269409.0,...,1488263.0,241740.0,144221.0,85219.0,184276.0,898739.0,304813.0,301130.0,254337.0,190167.0
2021-11-30,294440.0,240861.0,414526.0,338328.0,544713.0,367834.0,354691.0,323337.0,382814.0,272676.0,...,1501962.0,244923.0,145440.0,87406.0,186584.0,909127.0,307362.0,304551.0,256295.0,181762.0
2021-12-31,297088.0,243257.0,417568.0,341091.0,550440.0,371577.0,357374.0,326300.0,385893.0,275743.0,...,1512967.0,248236.0,147061.0,89012.0,188240.0,919617.0,308740.0,307136.0,259127.0,175064.0
2022-01-31,299729.0,245957.0,420270.0,343829.0,554475.0,374726.0,359804.0,328851.0,389127.0,278739.0,...,1522476.0,251014.0,148995.0,89909.0,190385.0,929894.0,310763.0,309673.0,261888.0,176039.0


In [15]:
#pull sample - 'Chicago' {vector}
chicago = dataset['Chicago']

In [19]:
type(chicago)

pandas.core.series.Series

In [36]:
# load dataset
# init setup
pip install pycaret-ts-alpha

SyntaxError: invalid syntax (39525545.py, line 3)

In [45]:
#import pycaret time series and load 'Chicago'
from pycaret.time_series import *
s = setup(data=chicago, fh = 3, fold = 5, session_id = 32)

ModuleNotFoundError: No module named 'pycaret.utils.time_series'; 'pycaret.utils' is not a package

In [ ]:
#compare models
best = compare_models()

In [ ]:
#plot best model forcast
plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 24})

In [ ]:
#plot best model diagnostics
plot_model(best, plot = 'diagnostics')

In [ ]:
#?
plot_model(best, plot = 'insample')

In [ ]:
# finalize model
final_best = finalize_model(best)
predict_model(best, fh = 24)

In [ ]:
#save test run / evaluate and discuss with team
save_model(final_best, 'my_final_best_model')

In [ ]:
#print details
loaded_model = load_model('my_final_best_model')
print(loaded_model)